# Climate Protection Actions Around the World

Looking at climate change performance index via https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

Goal: Produce interactive map that allows for comparison of climate change index across countries as well as more information about what specifically is contributing to to climate change and what needs to be altered.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

## Data Importing, Cleaning, and Preparation

### Loading Base Data

Base data contains the 2022 Climate Change Performance Index results from  https://en.wikipedia.org/wiki/Climate_Change_Performance_Index#2022_results

In [2]:
# Remove all rows with NaN values because all information is necessary
ccpi = pd.read_csv("data/ccpi.csv").dropna()
ccpi.head()

,rank,country,iso,score
0,1,Denmark,DNK,76.67
1,2,Sweden,SWE,74.22
2,3,Norway,NOR,73.29
3,4,United Kingdom,GBR,73.09
4,5,Morocco,MAR,71.60


### Importing Emissions Data

Emissions data contains information about different gasseous emissions ranging from 1750 to 2020 and is from https://www.kaggle.com/datasets/danielrpdias/co2-and-greenhouse-gas-emissions

In [3]:
# Drop rows where an ISO Code is not provided as they cannot be used on visualization
emissions = pd.read_csv("data/emissions.csv").dropna(subset=["iso_code"])
emissions.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,...,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,...,NaN,NaN,NaN,NaN,NaN,7752117.0,9.421400e+09,NaN,NaN,NaN
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7840151.0,9.692280e+09,NaN,NaN,NaN
3,AFG,Afghanistan,1952,0.092,NaN,0.0,0.000,NaN,0.012,NaN,...,NaN,NaN,NaN,NaN,NaN,7935996.0,1.001732e+10,NaN,NaN,NaN
4,AFG,Afghanistan,1953,0.106,NaN,16.0,0.015,NaN,0.013,NaN,...,NaN,NaN,NaN,NaN,NaN,8039684.0,1.063052e+10,NaN,NaN,NaN


In [4]:
# Query only most recent available year of emissions
# Reset index and drop columns where all values are NaN
emissions_last = emissions[emissions["year"] == max(emissions["year"])].reset_index(drop=True).dropna(axis="columns", how="all")
emissions_last.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,population
0,AFG,Afghanistan,2020,12.160,NaN,0.11,0.014,NaN,0.312,NaN,...,20.150,103.833,NaN,0.01,0.01,NaN,0.01,0.02,NaN,38928341.0
1,ALB,Albania,2020,4.535,NaN,-6.76,-0.329,NaN,1.576,NaN,...,16.602,182.453,NaN,0.05,0.01,NaN,0.01,0.03,NaN,2877800.0
2,DZA,Algeria,2020,154.995,NaN,-6.99,-11.646,NaN,3.535,NaN,...,2062.656,1590.074,NaN,0.53,0.01,3.31,0.84,0.27,NaN,43851043.0
3,AND,Andorra,2020,0.466,NaN,-7.08,-0.036,NaN,6.035,NaN,...,NaN,15.071,NaN,NaN,NaN,NaN,NaN,0.00,NaN,77265.0
4,AGO,Angola,2020,22.198,NaN,-5.49,-1.289,NaN,0.675,NaN,...,40.780,324.439,NaN,0.06,NaN,1.62,0.02,0.05,NaN,32866267.0


In [5]:
list(emissions_last.columns)

['iso_code',
 'country',
 'year',
 'co2',
 'consumption_co2',
 'co2_growth_prct',
 'co2_growth_abs',
 'trade_co2',
 'co2_per_capita',
 'consumption_co2_per_capita',
 'share_global_co2',
 'cumulative_co2',
 'share_global_cumulative_co2',
 'coal_co2',
 'cement_co2',
 'flaring_co2',
 'gas_co2',
 'oil_co2',
 'other_industry_co2',
 'cement_co2_per_capita',
 'coal_co2_per_capita',
 'flaring_co2_per_capita',
 'gas_co2_per_capita',
 'oil_co2_per_capita',
 'other_co2_per_capita',
 'trade_co2_share',
 'share_global_cement_co2',
 'share_global_coal_co2',
 'share_global_flaring_co2',
 'share_global_gas_co2',
 'share_global_oil_co2',
 'share_global_other_co2',
 'cumulative_cement_co2',
 'cumulative_coal_co2',
 'cumulative_flaring_co2',
 'cumulative_gas_co2',
 'cumulative_oil_co2',
 'cumulative_other_co2',
 'share_global_cumulative_cement_co2',
 'share_global_cumulative_coal_co2',
 'share_global_cumulative_flaring_co2',
 'share_global_cumulative_gas_co2',
 'share_global_cumulative_oil_co2',
 'share_

In [6]:
# Query only most recent available year of greenhouse gas emissions
emissions_ghg = emissions[emissions["year"] == 2016].reset_index(drop=True).dropna(axis="columns", how="all")
emissions_ghg.head()

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,AFG,Afghanistan,2016,6.745,NaN,-14.67,-1.160,NaN,0.191,NaN,...,2.730,78.17,2.209,9.13,0.258,35383028.0,6.429743e+10,35.737,1010.014,0.556
1,ALB,Albania,2016,4.492,5.502,-0.08,-0.004,1.01,1.556,1.906,...,3.184,3.25,1.126,1.26,0.437,2886427.0,3.148182e+10,39.446,13668.121,1.148
2,DZA,Algeria,2016,148.929,NaN,-0.50,-0.754,NaN,3.673,NaN,...,5.181,49.44,1.219,11.91,0.294,40551398.0,5.746962e+11,616.819,15210.944,1.155
3,AND,Andorra,2016,0.469,NaN,0.79,0.004,NaN,6.068,NaN,...,1.299,0.05,0.649,0.00,0.000,77295.0,NaN,NaN,NaN,NaN
4,AGO,Angola,2016,31.404,NaN,-6.60,-2.217,NaN,1.089,NaN,...,6.245,51.61,1.789,21.00,0.728,28842482.0,1.783339e+11,104.001,3605.892,0.703


### Importing Energy Data

Energy data contains information about energy sources and usage for each country and comes from https://github.com/owid/energy-data

In [7]:
energy = pd.read_csv("data/energy.csv").dropna(subset=["iso_code"])

# Remove full world data and continents (except Australia)
energy = energy[
    (energy["country"] != "World") & 
    (energy["country"] != "Africa") & 
    (energy["country"] != "Europe")
]

energy.head()

,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_consumption,solar_elec_per_capita,solar_energy_per_capita,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
list(energy.columns)

['iso_code',
 'country',
 'year',
 'coal_prod_change_pct',
 'coal_prod_change_twh',
 'gas_prod_change_pct',
 'gas_prod_change_twh',
 'oil_prod_change_pct',
 'oil_prod_change_twh',
 'energy_cons_change_pct',
 'energy_cons_change_twh',
 'biofuel_share_elec',
 'biofuel_elec_per_capita',
 'biofuel_cons_change_pct',
 'biofuel_share_energy',
 'biofuel_cons_change_twh',
 'biofuel_consumption',
 'biofuel_cons_per_capita',
 'carbon_intensity_elec',
 'coal_share_elec',
 'coal_cons_change_pct',
 'coal_share_energy',
 'coal_cons_change_twh',
 'coal_consumption',
 'coal_elec_per_capita',
 'coal_cons_per_capita',
 'coal_production',
 'coal_prod_per_capita',
 'electricity_demand',
 'biofuel_electricity',
 'coal_electricity',
 'fossil_electricity',
 'gas_electricity',
 'hydro_electricity',
 'nuclear_electricity',
 'oil_electricity',
 'other_renewable_exc_biofuel_electricity',
 'other_renewable_electricity',
 'renewables_electricity',
 'solar_electricity',
 'wind_electricity',
 'electricity_generation'

In [9]:
energy_last = energy[energy["year"] == max(energy["year"])-1].reset_index(drop=True).dropna(axis="columns", how="all")
energy_last

,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_consumption,solar_elec_per_capita,solar_energy_per_capita,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.028,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN
1,ALB,Albania,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.425,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN
2,DZA,Algeria,2020,NaN,NaN,-6.321,-54.959,-10.465,-78.293,-8.149,...,1.480,15.507,33.757,0.013,-20.202,0.003,-0.005,0.020,0.228,0.45
3,ASM,American Samoa,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN
4,AGO,Angola,2020,NaN,NaN,0.000,NaN,-6.593,-52.962,NaN,...,NaN,0.609,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,VEN,Venezuela,2020,182.660,3.822,-26.684,-68.248,-41.254,-223.701,-26.052,...,0.031,0.352,1.091,0.085,NaN,0.000,0.000,0.000,3.165,0.00
206,VNM,Vietnam,2020,4.808,14.526,-11.571,-11.401,-12.395,-16.495,-1.857,...,23.321,50.340,239.587,0.342,60.557,0.000,0.001,0.002,7.397,0.02
207,YEM,Yemen,2020,NaN,NaN,1.283,0.013,0.215,0.102,NaN,...,NaN,16.764,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN
208,ZMB,Zambia,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.159,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN


### Feature Engineering

In [10]:
# Generate primary emissions source column
primary_source = []
primary_source_num = [] # Must include numeric representation because go.Choropleth does not allow for discrete/categorical color scales

for index, row in emissions_last.iterrows():
    sources = ["coal_co2", "gas_co2", "flaring_co2", "oil_co2", "cement_co2", "other_industry_co2"]
    
    # Generate dictionaries of values enumerations
    values = {key: row[key] for key in sources}
    nums = {key: i for i, key in enumerate(sources)}
    
    # Append data
    primary_source.append(max(values, key = values.get)[:-4].title())
    primary_source_num.append(nums.get(primary_source[-1].lower() + "_co2"))
    
emissions_last["primary_co2_source"] = primary_source
emissions_last["primary_co2_source_num"] = primary_source_num

### Joining Data Sources

In [11]:
ccpi_emissions = ccpi.loc[:, ccpi.columns != "country"].merge(right=emissions_last, how="right", left_on="iso", right_on="iso_code").drop("iso", axis=1)
ccpi_emissions.dropna(axis="columns", how="all", inplace=True)
# Remove full world from dataset to avoid scaling issues
ccpi_emissions = ccpi_emissions[(ccpi_emissions["country"] != "World")]
# Replace NaN values with "No Data" so that they can still be displayed in different hover texts
ccpi_emissions = ccpi_emissions.fillna("No Data")
ccpi_emissions.head()

,rank,score,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,...,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,population,primary_co2_source,primary_co2_source_num
0,No Data,No Data,AFG,Afghanistan,2020,12.16,No Data,0.11,0.014,No Data,...,No Data,0.01,0.01,No Data,0.01,0.02,No Data,38928341.0,Coal,0
1,No Data,No Data,ALB,Albania,2020,4.535,No Data,-6.76,-0.329,No Data,...,No Data,0.05,0.01,No Data,0.01,0.03,No Data,2877800.0,Oil,3
2,51.0,39.91,DZA,Algeria,2020,154.995,No Data,-6.99,-11.646,No Data,...,No Data,0.53,0.01,3.31,0.84,0.27,No Data,43851043.0,Gas,1
3,No Data,No Data,AND,Andorra,2020,0.466,No Data,-7.08,-0.036,No Data,...,No Data,No Data,No Data,No Data,No Data,0.0,No Data,77265.0,Coal,0
4,No Data,No Data,AGO,Angola,2020,22.198,No Data,-5.49,-1.289,No Data,...,No Data,0.06,No Data,1.62,0.02,0.05,No Data,32866267.0,Coal,0


### Useful Function Preparation

In [12]:
# Functions to generate color breakpoints (top 10% and bottom 10% are extremes instead of max and min)
def get_max_bp(data, col, quantile=0.9):
    return data[data[col] != "No Data"][col].quantile(quantile)

def get_min_bp(data, col, quantile=0.1):
    return data[data[col] != "No Data"][col].quantile(quantile)

In [13]:
def to_log(x):
    if(isinstance(x, str)):
        return x
    else:
        return np.log(x) if x > 0 else np.log(0.01)

## Climate Change Protection Index

In [14]:
# Setting up plotting variables
locations = ccpi_emissions["iso_code"]

hovertemplate = "<br>".join([
    "%{customdata[0]} (%{customdata[1]})", 
    "CCPI Score: %{customdata[2]}",
    "CCPI Rank: %{customdata[3]}",
    "<extra></extra>"
])

customdata = ccpi_emissions[[
    "country", 
    "iso_code", 
    "score", 
    "rank"
]]

In [29]:
ccpi_fig = go.Figure(
    data=[
        go.Choropleth( 
            locations=locations, 
            z=ccpi_emissions["score"],  # Color values
            zmin=get_min_bp(ccpi_emissions, "score"),
            zmax=get_max_bp(ccpi_emissions, "score"),
            # Setting template for custom data in hover text
            hovertemplate=hovertemplate,
            colorscale="YlOrRd",
            reversescale=True,
            colorbar_title="Climate<br>Protection<br>Score",
            marker_line_width=0.1,
            # Custom data that will be included in hover text
            customdata=customdata
        )
    ],
    layout=go.Layout(
        width=950,
        height=500,
        title_text="Climate Change Protection Index",
        margin={"r":0,"t":50,"l":10,"b":0},
        geo={
            "projection_type": "natural earth",
            "showcoastlines": False,
            
        }
    )
)

ccpi_fig.show()
ccpi_fig.write_image("images/ccpi.png")
ccpi_fig.write_html("html_maps/ccpi.html")

## Greenhouse Gas Emissions

In [16]:
# Setting up plotting variables
locations_ghg = emissions_ghg["iso_code"]

hovertemplate_ghg = "<br>".join([
    "%{customdata[0]} (%{customdata[1]})", 
    "Greenhouse Gas Emissions per Capita: %{customdata[2]}",
    "Methane Emissions per Capita: %{customdata[3]}",
    "Nitrous Oxide Emissions per Capita: %{customdata[4]}",
    "CO<sub>2</sub> Emissions per Capita: %{customdata[5]}", 
    "<extra></extra>"
])

customdata_ghg = emissions_ghg[[
    "country", 
    "iso_code", 
    "ghg_per_capita", 
    "methane_per_capita",
    "nitrous_oxide_per_capita", 
    "co2_per_capita"
]]

In [28]:
ghg_fig = go.Figure(
    data=[
        # Need a new trace for each of the different data sets so that view can be properly updated
        go.Choropleth( 
            locations=locations_ghg, 
            z=emissions_ghg["ghg_per_capita"],
            zmin=get_min_bp(emissions_ghg, "ghg_per_capita"),
            zmax=get_max_bp(emissions_ghg, "ghg_per_capita"),
            hovertemplate=hovertemplate_ghg,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="GHG<br>Emissions<br>per Capita",
            colorbar=dict(
                ticksuffix=" tons",
                showticksuffix="all"
            ),
            marker_line_width=0.1,
            customdata=customdata_ghg,
            visible=True
        ),
        go.Choropleth( 
            locations=locations_ghg, 
            z=emissions_ghg["methane_per_capita"],
            zmin=get_min_bp(emissions_ghg, "methane_per_capita"),
            zmax=get_max_bp(emissions_ghg, "methane_per_capita"),
            hovertemplate=hovertemplate_ghg,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="Methane<br>Emissions<br>per Capita",
            colorbar=dict(
                ticksuffix=" tons",
                showticksuffix="all"
            ),
            marker_line_width=0.1,
            customdata=customdata_ghg,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_ghg, 
            z=emissions_ghg["nitrous_oxide_per_capita"],
            zmin=get_min_bp(emissions_ghg, "nitrous_oxide_per_capita"),
            zmax=get_max_bp(emissions_ghg, "nitrous_oxide_per_capita"),
            hovertemplate=hovertemplate_ghg,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="Nitrous Oxide<br>Emissions<br>per Capita",
            colorbar=dict(
                ticksuffix=" tons",
                showticksuffix="all"
            ),
            marker_line_width=0.1,
            customdata=customdata_ghg,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_ghg, 
            z=emissions_ghg["co2_per_capita"],
            zmin=get_min_bp(emissions_ghg, "co2_per_capita"),
            zmax=get_max_bp(emissions_ghg, "co2_per_capita"),
            hovertemplate=hovertemplate_ghg,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="CO<sub>2</sub><br>Emissions<br>per Capita",
            colorbar=dict(
                ticksuffix=" tons",
                showticksuffix="all"
            ),
            marker_line_width=0.1,
            customdata=customdata_ghg,
            visible=False
        )
    ],
    layout=go.Layout(
        width=950,
        height=500,
        title_text="Greenhouse Gas (GHG) Emissions",
        margin={"r":0,"t":50,"l":10,"b":0},
        geo={
            "projection_type": "natural earth",
            "showcoastlines": False,
        },
        updatemenus=[
            # Add dropdown menu that will allow for selection of data
            dict(
                buttons=[
                    dict(
                        args=[{"visible":(True, False, False, False)}],
                        label="Greenhouse Gas Emissions per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, True, False, False)}],
                        label="Methane Emissions per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, True, False)}],
                        label="Nitrous Oxide Emissions per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, True)}],
                        label="CO<sub>2</sub> Emissions per Capita",
                        method="update"
                    )
                ],
                direction="down",
                pad={"r":10, "t":0},
                x=0.425,
                xanchor="left",
                y=1.1,
                yanchor="top"
            )
        ]
    )
)

ghg_fig.show()
ghg_fig.write_image("images/ghg.png")
ghg_fig.write_html("html_maps/ghg.html")

## CO<sub>2</sub> Emissions

In [18]:
# Setting up plotting variables
locations_co2 = ccpi_emissions["iso_code"]

hovertemplate_co2 = "<br>".join([
    "%{customdata[0]} (%{customdata[1]})", 
    "CO<sub>2</sub> Emissions per Capita: %{customdata[2]}", 
    "Global Share of CO<sub>2</sub> Emissions: %{customdata[3]}",
    "Percent Growth of CO<sub>2</sub> Emissions: %{customdata[4]}",
    "Primary CO<sub>2</sub> Emissions Source: %{customdata[5]}",
    "<extra></extra>"
])

customdata_co2 = ccpi_emissions[[
    "country", 
    "iso_code", 
    "co2_per_capita", 
    "share_global_co2", 
    "co2_growth_prct",
    "primary_co2_source"
]]

In [27]:
co2_fig = go.Figure(
    data=[
        # Need a new trace for each of the different data sets so that view can be properly updated
        go.Choropleth( 
            locations=locations_co2, 
            z=ccpi_emissions["co2_per_capita"],
            zmin=get_min_bp(ccpi_emissions, "co2_per_capita"),
            zmax=get_max_bp(ccpi_emissions, "co2_per_capita"),
            hovertemplate=hovertemplate_co2,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="CO<sub>2</sub><br>Emissions<br>per Capita",
            colorbar=dict(
                ticksuffix=" tons"
            ),
            marker_line_width=0.1,
            customdata=customdata_co2,
            visible=True
        ),
        go.Choropleth( 
            locations=locations_co2, 
            z=ccpi_emissions["share_global_co2"],
            zmin=0,
            zmax=5,
            hovertemplate=hovertemplate_co2,
            colorscale="YlOrRd",
            reversescale=False,
            colorbar_title="Share of<br>Global CO<sub>2</sub><br>Emissions",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8805;",
                #showtickprefix="last"
            ),
            marker_line_width=0.1,
            customdata=customdata_co2,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_co2, 
            z=ccpi_emissions["co2_growth_prct"],
            zmid=0,
            zmin=-10,
            zmax=10,
            hovertemplate=hovertemplate_co2,
            colorscale="RdYlGn",
            reversescale=True,
            colorbar_title="Growth of<br>Global CO<sub>2</sub><br>Emissions",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_co2,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_co2, 
            z=ccpi_emissions["primary_co2_source_num"],
            colorscale="Sunset",
            # Plotly does not allow for categorical go.Choropleth color scales, so continuous diverging scale is used.
            colorbar=dict(
                tickvals=[0, 1, 2, 3],
                ticktext=["Coal", "Gas", "Flaring", "Oil"]
            ),
            colorbar_title="Primary CO<sub>2</sub><br>Emissions<br>Source",
            hovertemplate=hovertemplate_co2,
            marker_line_width=0.1,
            customdata=customdata_co2,
            visible=False
        )
    ],
    layout=go.Layout(
        width=950,
        height=500,
        title_text="CO<sub>2</sub> Emissions",
        margin={"r":0,"t":50,"l":10,"b":0},
        geo={
            "projection_type": "natural earth",
            "showcoastlines": False,
            
        },
        updatemenus=[
            # Add dropdown menu that will allow for selection of data
            dict(
                buttons=[
                    dict(
                        args=[{"visible":(True, False, False, False)}],
                        label="CO<sub>2</sub> Emissions per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, True, False, False)}],
                        label="Share of Global CO<sub>2</sub> Emissions",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, True, False)}],
                        label="Percent Growth of CO<sub>2</sub> Emissions",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, True)}],
                        label="Primary Source of CO<sub>2</sub> Emissions",
                        method="update"
                    )
                    
                ],
                direction="down",
                pad={"r":10, "t":0},
                x=0.22,
                xanchor="left",
                y=1.1,
                yanchor="top"
            )
        ]
    )
)

co2_fig.show()
co2_fig.write_image("images/co2.png")
co2_fig.write_html("html_maps/co2.html")

## Energy Sources

In [20]:
list(energy.columns)

['iso_code',
 'country',
 'year',
 'coal_prod_change_pct',
 'coal_prod_change_twh',
 'gas_prod_change_pct',
 'gas_prod_change_twh',
 'oil_prod_change_pct',
 'oil_prod_change_twh',
 'energy_cons_change_pct',
 'energy_cons_change_twh',
 'biofuel_share_elec',
 'biofuel_elec_per_capita',
 'biofuel_cons_change_pct',
 'biofuel_share_energy',
 'biofuel_cons_change_twh',
 'biofuel_consumption',
 'biofuel_cons_per_capita',
 'carbon_intensity_elec',
 'coal_share_elec',
 'coal_cons_change_pct',
 'coal_share_energy',
 'coal_cons_change_twh',
 'coal_consumption',
 'coal_elec_per_capita',
 'coal_cons_per_capita',
 'coal_production',
 'coal_prod_per_capita',
 'electricity_demand',
 'biofuel_electricity',
 'coal_electricity',
 'fossil_electricity',
 'gas_electricity',
 'hydro_electricity',
 'nuclear_electricity',
 'oil_electricity',
 'other_renewable_exc_biofuel_electricity',
 'other_renewable_electricity',
 'renewables_electricity',
 'solar_electricity',
 'wind_electricity',
 'electricity_generation'

In [35]:
# Setting up plotting variables
locations_energy = energy_last["iso_code"]

hovertemplate_energy = "<br>".join([
    "%{customdata[0]} (%{customdata[1]})", 
    "Share of Energy Consumption that is Renewable: %{customdata[2]}", 
    "Energy Consumed per Capita: %{customdata[9]}",
    "Share of Energy Consumption from Nuclear Power: %{customdata[3]}",
    "Share of Energy Consumption from Hydropower: %{customdata[4]}",
    "Share of Energy Consumption from Solar Power: %{customdata[5]}",
    "Share of Energy Consumption from Wind Power: %{customdata[6]}",
    "Share of Energy Consumption from Biofuel: %{customdata[7]}",
    "Share of Energy Consumption from Other Renewable Sources: %{customdata[8]}",
    "<extra></extra>"
])

customdata_energy = energy_last[[
    "country", 
    "iso_code", 
    "renewables_share_energy", 
    "nuclear_share_energy", 
    "hydro_share_energy",
    "solar_share_energy",
    "wind_share_energy",
    "biofuel_share_energy",
    "other_renewables_share_energy",
    "energy_per_capita"
]]

In [36]:
energy_fig = go.Figure(
    data=[
        # Need a new trace for each of the different data sets so that view can be properly updated
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["renewables_share_energy"],
            zmin=0,
            zmax=100,
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Renewables",
            colorbar=dict(
                ticksuffix="%"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=True
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["energy_per_capita"],
            zmin=get_min_bp(energy_last, "energy_per_capita"),
            zmax=get_max_bp(energy_last, "energy_per_capita"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Energy<br>Consumed<br>per Capita",
            colorbar=dict(
                ticksuffix=" kWh",
                showticksuffix="all",
                #tickprefix="&#8805;",
                #showtickprefix="last"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["nuclear_share_energy"],
            zmin=get_min_bp(energy_last, "nuclear_share_energy"),
            zmax=get_max_bp(energy_last, "nuclear_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Nuclear",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8805;",
                #showtickprefix="last"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["hydro_share_energy"],
            zmin=get_min_bp(energy_last, "hydro_share_energy"),
            zmax=get_max_bp(energy_last, "hydro_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Hydro",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["solar_share_energy"],
            zmin=get_min_bp(energy_last, "solar_share_energy"),
            zmax=get_max_bp(energy_last, "solar_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Solar",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["wind_share_energy"],
            zmin=get_min_bp(energy_last, "wind_share_energy"),
            zmax=get_max_bp(energy_last, "wind_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Wind",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["biofuel_share_energy"],
            zmin=get_min_bp(energy_last, "biofuel_share_energy"),
            zmax=get_max_bp(energy_last, "biofuel_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Biofuel",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        ),
        go.Choropleth( 
            locations=locations_energy, 
            z=energy_last["other_renewables_share_energy"],
            zmin=get_min_bp(energy_last, "other_renewables_share_energy"),
            zmax=get_max_bp(energy_last, "other_renewables_share_energy"),
            hovertemplate=hovertemplate_energy,
            colorscale="Greens",
            reversescale=False,
            colorbar_title="Share of<br>Consumption<br>from Other<br>Renewables",
            colorbar=dict(
                ticksuffix="%",
                showticksuffix="all",
                #tickprefix="&#8804;",
                #showtickprefix="first"
            ),
            marker_line_width=0.1,
            customdata=customdata_energy,
            visible=False
        )
    ],
    layout=go.Layout(
        width=950,
        height=500,
        title_text="Energy Sources",
        margin={"r":0,"t":50,"l":10,"b":0},
        geo={
            "projection_type": "natural earth",
            "showcoastlines": False,
            
        },
        updatemenus=[
            # Add dropdown menu that will allow for selection of data
            dict(
                buttons=[
                    dict(
                        args=[{"visible":(True, False, False, False, False, False, False, False)}],
                        label="Share of Energy Consumption from Renewable Sources",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, True, False, False, False, False, False, False)}],
                        label="Energy Consumption per Capita",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, True, False, False, False, False, False)}],
                        label="Share of Energy Consumption from Nuclear Power",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, True, False, False, False, False)}],
                        label="Share of Energy Consumption from Hydropower",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, False, True, False, False, False)}],
                        label="Share of Energy Consumption from Solar Power",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, False, False, True, False, False)}],
                        label="Share of Energy Consumption from Wind Power",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, False, False, False, True, False)}],
                        label="Share of Energy Consumption from Biofuel",
                        method="update"
                    ),
                    dict(
                        args=[{"visible":(False, False, False, False, False, False, False, True)}],
                        label="Share of Energy Consumption from Other Renewables",
                        method="update"
                    )
                    
                ],
                direction="down",
                pad={"r":10, "t":0},
                x=0.23,
                xanchor="left",
                y=1.1,
                yanchor="top"
            )
        ]
    )
)

energy_fig.show()
energy_fig.write_image("images/energy.png")
energy_fig.write_html("html_maps/energy.html")